<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# BlackHoles@Home Tutorial: Compiling the `BOINC` libraries

## Author: Leo Werneck

## This tutorial notebook demonstrates how to compile the `BOINC` libraries on different platforms

## Introduction:

The [BlackHoles@Home](http://blackholesathome.net/) project allows users to volunteer CPU time so a large number of binary black holes simulations can be performed. The objective is to create a large catalog of [gravitational waveforms](https://en.wikipedia.org/wiki/Gravitational_wave), which can be used by observatories such as [LIGO](https://www.ligo.org), [VIRGO](https://www.virgo-gw.eu), and, in the future, [LISA](https://lisa.nasa.gov) in order to infer what was the source of a detected gravitational wave.

BlackHoles@Home is destined to run on the [BOINC](https://boinc.berkeley.edu) infrastructure (alongside [Einstein@Home](https://einsteinathome.org/) and [many other great projects](https://boinc.berkeley.edu/projects.php)), enabling anyone with a computer to contribute to the construction of the largest numerical relativity gravitational wave catalogs ever produced.

### Additional Reading Material:

* [BOINC's Wiki page](https://boinc.berkeley.edu/trac/wiki)
* [Debian's Wiki BOINC server guide](https://wiki.debian.org/BOINC/ServerGuide/Initialisation)

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This tutorial compiles the `BOINC` libraries. It will also install all needed dependencies. We test everything is okay by compiling and running a simple "Hello World!" application that makes use of the `BOINC` libraries. This tutorial is organized as follows:

1. [Step 1](#loading_python_nrpy_modules): Loading necessary Python/NRPy+ modules
1. [Step 2](#compilation_script): A simple script to compile the `BOINC` libraries
1. [Step 3](#compilating_boinc_libraries): Compiling the `BOINC` libraries
1. [Step 4](#compilation_test): Compiling a simple program
1. [Step 5](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='loading_python_nrpy_modules'></a>

# Step 1: Loading needed Python/NRPy+ modules \[Back to [top](#toc)\]
$$\label{loading_python_nrpy_modules}$$

We start by loading the necessary Python/NRPy+ moduels used by this tutorial notebook. We also set up the `BOINC` directory path (the default path is the current working directory).

In [1]:
# Step 1: Import necessary modules - set directories.
# Step 1.a: Import required Python modules
import os,sys,shutil

# Step 1.b: Add NRPy's root directory to the sys.path()
sys.path.append("..")

# Step 1.c: Load NRPy+'s command line helper module
import cmdline_helper as cmd

# Step 1.d: Set directories needed by this tutorial notebook
boinc_base_dir = "/Users/werneck/bhah" # os.getcwd()
boinc_root_dir = os.path.join(boinc_base_dir,"boinc")
boinc_lib_dir  = os.path.join(boinc_root_dir,"lib")
boinc_api_dir  = os.path.join(boinc_root_dir,"api")
boinc_zip_dir  = os.path.join(boinc_root_dir,"zip")

<a id='compilation_script'></a>

# Step 2: A simple script to compile the `BOINC` libraries \[Back to [top](#toc)\]
$$\label{compilation_script}$$

We will now write a simple `bash` script that will take care of downloading the `BOINC` source code and compiling the `BOINC` libraries.

The script performs the following tasks:

1. Install the necessary dependencies.
1. Download the [`BOINC` source code](https://github.com/BOINC/boinc) (if necessary).
1. Compile the `BOINC` core, api, and zip libraries.

The `BOINC` library require the following packages:

1. git
1. make
1. m4
1. autoconf
1. libtool
1. A C++ compiler

In Linux, we will be using the [GNU compilers](https://gcc.gnu.org) installed using [`aptitude`](http://manpages.ubuntu.com/manpages/bionic/man8/aptitude-curses.8.html) in Ubuntu and Debian. In Mac OS X, we will be using the [LLVM compilers](https://llvm.org) installed using [`homebrew`](https://brew.sh).

In [2]:
# Step 2: Write down a script to compile the BOINC libraries
# Step 2.a: Check the platform. This will adjust the way we install the dependencies.
platform = sys.platform
compiler_config = ""
if platform == "linux":
    # In Ubuntu/Debian, use aptitude to install the dependencies.
    # To avoid issues where the user does not have sudo priviledges,
    # such as in mybinder, we check if we find the command first.
    dependency_list = ["git","make","m4","autoconf","libtool","g++"]
    dependencies    = ""
    for dependency in dependency_list:
        path = shutil.which(dependency)
        if path is not None:
            dependencies += dependency+" "
        else:
            print(dependency+" found at "+path)
        
        if dependencies != "":
            install_dependencies = "sudo apt install -y "+dependencies
elif platform == "darwin":
    # In Mac OS X, use homebrew to install the dependencies.
    install_dependencies = "brew install git make m4 autoconf libtool llvm"
    # Because we use clang/clang++ instead of gcc/g++, we need
    # to specify additional compilation flags to help the build
    # script locate all the header files and libraries we will
    # need. The paths below assume that the llvm compilers are
    # in the default brew path /usr/local/opt. Adjust the
    # variable 'path_to_llvm' if that's not the case.
    path_to_llvm          = "/usr/local/opt/llvm/"
    path_to_clang         = os.path.join(path_to_llvm,"bin","clang")
    path_to_clangpp       = os.path.join(path_to_llvm,"bin","clang++")
    path_to_clang_include = os.path.join(path_to_llvm,"include")
    path_to_clang_library = os.path.join(path_to_llvm,"lib")
    CPPFLAGS              = "-I%s"%(path_to_clang_include)
    LDFLAGS               = "-L%s"%(path_to_clang_library)
    compiler_config       = "CC=%s CXX=%s OBJC=%s CPPFLAGS=\"%s\" LDFLAGS=\"%s\" "%(path_to_clang,
                                                                                    path_to_clangpp,
                                                                                    path_to_clang,
                                                                                    path_to_clang_include,
                                                                                    path_to_clang_library)
else:
    print("Error: platform %s is currently not supported."%platform)
    sys.exit(1)
    
with open("compile_BOINC_libraries.sh","w") as file:
    file.write(r"""#!/bin/bash

# Install all required software
%s

# Download BOINC repository (if necessary)
boincdir=%s
if [ -d "$boincdir" ]; then
  echo "BOINC directory found at $boincdir"
else
  echo "BOINC directory not found at $boincdir. Cloning from BOINC github repository..."
  git clone https://github.com/BOINC/boinc boinc
fi

# Now change directories to the BOINC directory
cd "$boincdir"

# Compile the core BOINC libraries
./_autosetup -f
%s./configure --disable-client --disable-manager --disable-server
make

# Now compile the boinc_zip library
cd %s
make
"""%(install_dependencies,boinc_root_dir,compiler_config,boinc_zip_dir))

<a id='compilating_boinc_libraries'></a>

# Step 3: Compiling the `BOINC` libraries \[Back to [top](#toc)\]
$$\label{compilating_boinc_libraries}$$

We now run the script above to compile the `BOINC` libraries. This tutorial notebook has been tested in the following systems:

1. Ubuntu 20.04 LTS.
1. Mac OS X Big Sur (version 11.2.3).

If all the dependencies are already installed, then the script should finish running in just a few minutes. If not, then it will download the dependencies and the execution time may vary depending on your internet connection speed.

In [3]:
# Step 3: Compile the BOINC libraries
# FIXME: I don't think this is the best way of doing this.
print("Now compiling BOINC libraries (can take a few minutes depending on your internet connection) ...")
status = os.system("source compile_BOINC_libraries.sh")
print("Finished compiling BOINC libraries" if status == 0 
      else "Error: Something went wrong when compiling the BOINC libraries.")

Now compiling BOINC libraries (can take a few minutes depending on your internet connection) ...
Finished compiling BOINC libraries


<a id='compilation_test'></a>

# Step 4: Compiling a simple program \[Back to [top](#toc)\]
$$\label{compilation_test}$$

We now compile a "Hello World!" program that makes use of the `BOINC` libraries. This will help us know if everything is okay with the compilation of the libraries.

In [4]:
# Step 3: Compiling a test program
# Step 3.a: Write down a "Hello World!" program that uses the BOINC libraries
outfile = "hello_boinc.cpp"
with open(outfile,"w") as file:
    file.write(r"""
// Core C++ input/output header
#include <iostream>

// BOINC API header
#include "boinc_api.h"

// Main program
int main(int argc, char** argv) {

  // Initialize the boinc environment
  int status = boinc_init();
  
  // Check everything is okay
  if( status != 0 ) {
    fprintf(stderr,"Error: boinc_init() returned a non-zero value: %d. Terminating.",status);
    boinc_finish(status);
  }
  
  // Print a message to the user
  printf("Hello BOINC!\n");
  
  // Terminate the program with no errors
  boinc_finish(0);
  
  // All done!
}
""")

# Step 3.b: Add the base nrpytutorial directory to sys.path()
import sys
sys.path.append('..')

# Step 3.c: Import the cmdline_helper NRPy+ module
import cmdline_helper as cmd

# Step 3.d: Prepare to compile
# Step 3.d.i: Set the C++ compiler flags
CXXFLAGS = "-I%s -I%s "%(boinc_api_dir,boinc_lib_dir)
LDFLAGS  = "-L%s -L%s -lboinc_api -lboinc "%(boinc_api_dir,boinc_lib_dir)

# Step 3.d.ii: Set the C++ compiler
if platform == 'linux':
    CXX_compiler = "g++ "
elif platform == 'darwin':
    CXX_compiler = "clang++ "
    CXXFLAGS    += "-I%s "%(path_to_clang_include)
    LDFLAGS     += "-L%s "%(path_to_clang_library)
else:
    print("Error: platform %s is currently not supported."%platform)
    sys.exit(1)

# Step 3.d.iii: Compile the program
CXX_compile_string = CXX_compiler+CXXFLAGS+"hello_boinc.cpp -o hello_boinc "+LDFLAGS
os.system(CXX_compile_string)

# Step 3.d.iv: Run the program
print("Running hello_boinc program...")
!./hello_boinc
print("All done!")

Running hello_boinc program...
Hello BOINC!
All done!


<a id='latex_pdf_output'></a>

# Step 5: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-BlackHolesAtHome-Compiling_the_BOINC_libraries.pdf](Tutorial-BlackHolesAtHome-Compiling_the_BOINC_libraries.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [7]:
!cp ../latex_nrpy_style.tplx .
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-BlackHolesAtHome-Compiling_the_BOINC_libraries")
!rm -f latex_nrpy_style.tplx

Created Tutorial-BlackHolesAtHome-Compiling_the_BOINC_libraries.tex, and
    compiled LaTeX file to PDF file Tutorial-BlackHolesAtHome-
    Compiling_the_BOINC_libraries.pdf
